In [ ]:
import utils

CONFIG_ELN = utils.get_aiidalab_eln_config()
openbis_url = "local.openbis.ch"
openbis_token = "$pat-admin-250611125344861x11B798860F11755C1C796FFB557F2CA0"
OPENBIS_SESSION, _ = utils.connect_openbis(openbis_url, openbis_token)

In [ ]:
obj = OPENBIS_SESSION.get_object("20250612080918237-1213")
obj.props["description"]

In [ ]:
obj.props.all()

In [ ]:
import utils

CONFIG_ELN = utils.get_aiidalab_eln_config()
openbis_url = "https://openbis-empa-lab205.ethz.ch/"
openbis_token = "$pat-dafa_admin-250604134543930xA579A4380CE9658576376E29B1098549"
OPENBIS_SESSION, _ = utils.connect_openbis(openbis_url, openbis_token)

## Create rich text property and assign it to an object

In [ ]:
property_type_dict = {
    "code": "complex_description",
    "label": "Complex description",
    "description": "Complex description",
    "dataType": "MULTILINE_VARCHAR",
    "managedInternally": False,
    "multiValue": False,
    "metaData": {"custom_widget": "Word Processor"},
}

prop_type = OPENBIS_SESSION.new_property_type(**property_type_dict).save()
obj_type = OPENBIS_SESSION.get_object_type("TEST_OBJECT")
obj_type.assign_property(
    OPENBIS_SESSION.get_property_type(code=property_type_dict["code"])
)

# You have to reload the session after modifying object types
OPENBIS_SESSION, _ = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

## Get complex_description context from openBIS

In [ ]:
OPENBIS_SESSION.get_object("20250409122357804-4460").props.all()["complex_description"]

## Write complex description into openBIS

In [ ]:
import requests
import json

# Assuming you have logged into openbis via pybis

obj = OPENBIS_SESSION.get_object(
    "20250409122357804-4460"
)  # Experiment or sample you want to update

file = "ase_geo.png"
#
url = (
    "https://local.openbis.ch/openbis/openbis/file-service/eln-lims?type=Files&sessionID="
    + OPENBIS_SESSION.token
)
files = {"file": open(file, "rb")}

# Upload figure into openBIS file-service. This figure is goign to be used in the rich text property
r = requests.post(
    url, files=files, verify=False
)  # verify=False is required if you use self-signed certificates

if r.ok:
    url = json.loads(r.text)["url"]  # url is a special url to uploaded image
    formatted = (
        "<p><strong>adasdadasdjandkjadaj</strong></p><p>&nbsp;"  ## Add boolean text
    )
    formatted += f"<figure class='image'><img src='{url}'/></figure>"  ## Add figure
    obj.props["complex_description"] = (
        formatted  # setting text with image to a example $document property
    )
    obj.save()

## Read content from openBIS

In [ ]:
from bs4 import BeautifulSoup

session_id = OPENBIS_SESSION.token  # Your active openBIS session
obj = OPENBIS_SESSION.get_object("20250409122357804-4460")
html = obj.props["complex_description"]

# Get image url from the html string
soup = BeautifulSoup(html, "html.parser")
img_tag = soup.find("img")
if img_tag:
    url = img_tag["src"]

# Combine base URL + path + sessionID
full_url = f"https://local.openbis.ch{url}?sessionID={session_id}"

# Download the file
response = requests.get(full_url, verify=False)

if response.status_code == 200:
    with open("downloaded_image.png", "wb") as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print(f"Failed to download: {response.status_code} - {response.text}")